# SLEAP Notebook:

## Imports:

In [12]:
from sleap.nn.config import *
from wrapper_sleap import *

## Converting .h5 files to pkg.slp files:

In [13]:
slp_wrapper = SleapWrapper(
    "/Users/marcusschou/Desktop/SDU/data_science/kurser_4/Speciale_Data_Science/SLEAP/data"
)

In [4]:
slp_wrapper.convert_to_slp(
    "/Users/marcusschou/Desktop/SDU/data_science/kurser_4/Speciale_Data_Science/data/h5_files/head_annotation_data_nat_updated_v2.h5",
    "/Users/marcusschou/Desktop/SDU/data_science/kurser_4/Speciale_Data_Science/data/skeletons/natskeleton_noposs.csv"
)

Total frames: 1566
Frames marked for removal: 354
Frames kept: 1212
New Annotations shape: (1212, 20, 2)
New Images shape: (1212, 1024, 1024, 1)


100%|██████████| 1212/1212 [00:03<00:00, 403.27it/s]


Video saved to: /Users/marcusschou/Desktop/SDU/data_science/kurser_4/Speciale_Data_Science/SLEAP/data/head_annotation_data_nat_updated_v2.mp4


100%|██████████| 1212/1212 [00:00<00:00, 44987.49it/s]


SLEAP labels file saved to: /Users/marcusschou/Desktop/SDU/data_science/kurser_4/Speciale_Data_Science/SLEAP/data/head_annotation_data_nat_updated_v2.slp


## Model config:

### Generating split indicies:

In [14]:
slp_filepath = "/Users/marcusschou/Desktop/SDU/data_science/kurser_4/Speciale_Data_Science/SLEAP/data/head_annotation_data_nat_updated_v2.slp"

train_indicies, test_indicies = slp_wrapper.split_indicies(slp_filepath, train_split_percentage=0.8, test_split_percentage=0.2)

In [42]:
RUN_NAME = "head_annotation_data_nat_updated_v2_run1_baseline"

In [43]:
BATCH_SIZE = 2
EPOCHS = 400

## Model configuration: (Single instance)

In [44]:
MODEL_NAME = "head_annotation_data_nat_updated_v2_config_baseline_medium_rf_single_run1_UNET.json"

train_config = TrainingJobConfig()

# Data config:
train_config.data = DataConfig(
    labels=LabelsConfig(
        split_by_inds=True,
        training_inds=train_indicies,
        validation_inds=test_indicies,
    ),
    preprocessing=PreprocessingConfig(
        input_scaling=1.0 
    ),
    instance_cropping=InstanceCroppingConfig()
)

# Model config
train_config.model = ModelConfig(
    backbone=BackboneConfig(
       unet=UNetConfig(
            max_stride=16,
            filters=24,
            output_stride=2,
            up_interpolate=True,
        )
    ),
    heads=HeadsConfig(
        single_instance=SingleInstanceConfmapsHeadConfig(
            sigma=2.5,
            output_stride=2
        )
    ) 
)

# Optimization config:
train_config.optimization = OptimizationConfig(
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    augmentation_config=AugmentationConfig(
        rotate=True,
        rotation_max_angle=15.0,
        rotation_min_angle=-15.0,
        scale=True,
        scale_max=1.2,
        scale_min=0.8,
        random_flip=True,
        flip_horizontal=True,
    ),
    hard_keypoint_mining=HardKeypointMiningConfig(),
    learning_rate_schedule=LearningRateScheduleConfig(
        plateau_min_delta=1e-06,
        min_learning_rate=1e-08
    ),
    early_stopping=EarlyStoppingConfig(
        plateau_min_delta=1e-08,
        plateau_patience=10
    )
)

# Outputs config:
train_config.outputs = OutputsConfig(
    checkpointing=CheckpointingConfig(
        best_model=True
    ),
    run_name=RUN_NAME,
    run_name_suffix=".single_instance",
    runs_folder="models",
    keep_viz_images=True,
    tensorboard=TensorBoardConfig(
        write_logs=True
    )
)

train_config.save_json(MODEL_NAME)

## Model configuration: (top-down)

### Centroid config:

In [22]:
MODEL_NAME = "head_annotation_data_nat_updated_v2_basline_centroid_run1_UNET.json"
ANCHOR_PART = "Nose"

train_config = TrainingJobConfig()

# Data config:
train_config.data = DataConfig(
    labels=LabelsConfig(
        validation_fraction=0.1
    ),
    preprocessing=PreprocessingConfig(
        input_scaling=0.5 
    ),
    instance_cropping=InstanceCroppingConfig(
        center_on_part=ANCHOR_PART
    )
)

# Model config
train_config.model = ModelConfig(
    backbone=BackboneConfig(
       unet=UNetConfig(
            max_stride=16,
            filters=24,
            output_stride=2,
            up_interpolate=True
        )
    ),
    heads=HeadsConfig(
        centroid=CentroidsHeadConfig(
            sigma=2.5,
            output_stride=2,
            anchor_part=ANCHOR_PART
        )
    ) 
)

# Optimization config:
train_config.optimization = OptimizationConfig(
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    augmentation_config=AugmentationConfig(),
    hard_keypoint_mining=HardKeypointMiningConfig(),
    learning_rate_schedule=LearningRateScheduleConfig(
        plateau_min_delta=1e-06,
        min_learning_rate=1e-08
    ),
    early_stopping=EarlyStoppingConfig(
        plateau_min_delta=1e-08,
        plateau_patience=10
    )
)

# Outputs config:
train_config.outputs = OutputsConfig(
    checkpointing=CheckpointingConfig(
        best_model=True
    ),
    run_name=RUN_NAME,
    run_name_suffix=".centroid",
    runs_folder="models",
    keep_viz_images=True,
    tensorboard=TensorBoardConfig(
        write_logs=True
    )
)

train_config.save_json(MODEL_NAME)

### Centered instance config:

In [40]:
MODEL_NAME = "head_annotation_data_nat_updated_v2_basline_centered_instance_run1_UNET.json"

train_config = TrainingJobConfig()

# Data config:
train_config.data = DataConfig(
    labels=LabelsConfig(
        split_by_inds=True,
        training_inds=train_indicies,
        validation_inds=test_indicies,
    ),
    preprocessing=PreprocessingConfig(
        input_scaling=1.0 
    ),
    instance_cropping=InstanceCroppingConfig(
        center_on_part=ANCHOR_PART
    )
)

# Model config
train_config.model = ModelConfig(
    backbone=BackboneConfig(
       unet=UNetConfig(
            max_stride=16,
            filters=24,
            output_stride=4,
            up_interpolate=True
        )
    ),
    heads=HeadsConfig(
        centered_instance=CenteredInstanceConfmapsHeadConfig(
            sigma=2.5,
            output_stride=4,
            anchor_part=ANCHOR_PART  
        )
    ) 
)

# Optimization config:
train_config.optimization = OptimizationConfig(
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    augmentation_config=AugmentationConfig(
        rotate=True,
        rotation_max_angle=15.0,
        rotation_min_angle=-15.0,
        scale=True,
        scale_max=1.2,
        scale_min=0.8,
        random_flip=True,
        flip_horizontal=True,
    ),
    hard_keypoint_mining=HardKeypointMiningConfig(),
    learning_rate_schedule=LearningRateScheduleConfig(
        plateau_min_delta=1e-06,
        min_learning_rate=1e-08
    ),
    early_stopping=EarlyStoppingConfig(
        plateau_min_delta=1e-08,
        plateau_patience=10
    )
)

# Outputs config:
train_config.outputs = OutputsConfig(
    checkpointing=CheckpointingConfig(
        best_model=True
    ),
    run_name=RUN_NAME,
    run_name_suffix=".centered_instance",
    runs_folder="models",
    keep_viz_images=True,
    tensorboard=TensorBoardConfig(
        write_logs=True
    )
)

train_config.save_json(MODEL_NAME)

### pretrained backbone:

In [45]:
MODEL_NAME = "head_annotation_data_nat_updated_v2_config_baseline_medium_rf_single_pretrained_backbone_run1_UNET.json"

train_config = TrainingJobConfig()

# Data config:
train_config.data = DataConfig(
    labels=LabelsConfig(
        split_by_inds=True,
        training_inds=train_indicies,
        validation_inds=test_indicies,
    ),
    preprocessing=PreprocessingConfig(
        input_scaling=1.0 
    ),
    instance_cropping=InstanceCroppingConfig()
)

# Model config
train_config.model = ModelConfig(
    backbone=BackboneConfig(
        pretrained_encoder=PretrainedEncoderConfig(
            encoder = "resnet50"
        )
    ),
    heads=HeadsConfig(
        single_instance=SingleInstanceConfmapsHeadConfig(
            sigma=2.5,
            output_stride=2
        )
    ) 
)

# Optimization config:
train_config.optimization = OptimizationConfig(
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    augmentation_config=AugmentationConfig(
        rotate=True,
        rotation_max_angle=15.0,
        rotation_min_angle=-15.0,
        scale=True,
        scale_max=1.2,
        scale_min=0.8,
        random_flip=True,
        flip_horizontal=True,
    ),
    hard_keypoint_mining=HardKeypointMiningConfig(),
    learning_rate_schedule=LearningRateScheduleConfig(
        plateau_min_delta=1e-06,
        min_learning_rate=1e-08
    ),
    early_stopping=EarlyStoppingConfig(
        plateau_min_delta=1e-08,
        plateau_patience=10
    )
)

# Outputs config:
train_config.outputs = OutputsConfig(
    checkpointing=CheckpointingConfig(
        best_model=True
    ),
    run_name=RUN_NAME,
    run_name_suffix=".single_instance",
    runs_folder="models",
    keep_viz_images=True,
    tensorboard=TensorBoardConfig(
        write_logs=True
    )
)

train_config.save_json(MODEL_NAME)

## Training:

1. **CLI Training:** config.json labels.slp
3. **CLI Inference:** sleap-track -m "models/my_model" --tracking.tracker simple -o "output_predictions.slp" "input_video.mp4"
4. **CLI Inference top-down:** sleap-track -m "models/centroid" -m "models/centered_instance" -o "output_predictions.slp" "input_video.mp4"

sleep-train centroid_json colab_.pkg.slp && sleep-train centered_instance_json colab_.pkg.slp